In [1]:
import sys
sys.path.insert(0, 'Resources/MagicCube/code/')
import matplotlib.pyplot as plt
import random as rd
from bitstring import BitArray
import math
import copy
import tensorflow as tf
import random

from cube import *
from utils import *

In [66]:
N = 3 #cube size

#Q-learning parameters
r = 0.05
gamma = 1 / (1 + r) #discount of the model
C = 1.
epsilon = 0.05
beta = 3./4 

In [105]:
def all_actions(N): #rotate by +90° / by -90° 
    actions = []
    c = Cube(N)
    for face_name in list(Cube.facedict.keys())[:3]: #the list is in the end ['U','D','F']
        for layer in range(c.N):
            for times in [1,-1]:
                actions.append([face_name,layer,times])
    return actions

def reward_cube(c):
    edges = computeEdges(c)
    corners = computeCorners(c)
    ncf = numCompleteFaces(c)
    nce = numCompleteEdges(c,edges)
    ncc = numCompleteCorners(c,corners)
    
    return -1 + 10*ncf + 2*nce + 3*ncc + 1000*(ncf == 6)

def state_cube(c):
    #determining the new state
    edges = computeEdges(c)
    corners = computeCorners(c)
    edges_state = []
    corners_state = []
    faces_state = []
#     for e in edges:
#         edges_state.append(e.isDone(c))
    for corner in corners:
        corners_state.append(corner.isDone(c))
    nFaces = 6
    for f in range(nFaces):
        faces_state.append(np.sum(c.stickers[f] != c.stickers[f,0,0]) == 0)
#     #conversion from binary list to int
#     e = BitArray(edges_state).uint
    c = BitArray(corners_state).uint
    f = BitArray(faces_state).uint
#     ncf = numCompleteFaces(c)
#     nce = numCompleteEdges(c,edges)
#     ncc = numCompleteCorners(c,corners)
#     return ncf,nce,ncc
    return c,f

def test_function_state_cube():
    c = Cube(3)
    print(state_cube(c))
    c.randomize(1)
    print(state_cube(c))

In [106]:
actions = all_actions(N) #rotate by +90° / by -90° 
nb_actions = len(actions)

In [117]:
c_init=Cube(3)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None,6*c_init.N**2])
act = tf.placeholder(tf.float32, shape=[nb_actions,None])
Q_ = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.zeros([6*c_init.N**2,nb_actions]))
b = tf.Variable(tf.zeros([nb_actions]))

sess.run(tf.global_variables_initializer())

Q = tf.matmul(x,W) + b
Qs = tf.matmul(Q,act)
    
loss_function = tf.reduce_mean(tf.square(tf.sub(Q_,Qs)))

train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(loss_function)

D = []

In [ ]:
def DQN(c_init,Tmax,nb_episodes, n_moves):
    
    done = 0

    for episode in range(nb_episodes):  
        
        s = copy.deepcopy(c_init)
        s.randomize(n_moves) #we randomize n_moves times in order to have a "well mixed" cube
        cum_reward = 0

        for i in range(Tmax):
            
            #transition 
            S = np.reshape(s.stickers,(1, 54))
            if(rd.random() > epsilon):
                a = np.argmax(Q.eval({x:S}))
            else:
                a = rd.randint(0,nb_actions-1)

            f,l,d = actions[a]
            s.move(f,l,d)            
            R = reward_cube(s)
            cum_reward += R
            D.append([S, a, R, np.reshape(s.stickers,(1, 54))])
            
            if numCompleteFaces(s) == 6:
                done += 1
                break
                
#             print(D[0])
            Dshuf = D
            random.shuffle(Dshuf)
            batch = np.array(Dshuf[:100])
            
            tts = np.empty([0,1])
            
            for i in range(len(batch)):
                
                faces_done = np.sum([np.sum([batch[i][-1][0][f*9+j] != batch[i][-1][0][f*9] for j in range(9)]) == 0 for f in range(6)])
                
                if faces_done == 6:
                    tt = batch[i][-2]
                else:
                    tt = batch[i][-2] + gamma*max(Q.eval({x:batch[i][-1]})[0])
             
                tts = np.append(tts,tt)
                
            tts = np.reshape(tts,(-1, 1))
            
            action = np.reshape([[i == batch[j,1] for i in range(nb_actions)] for j in range(len(batch))], (-1,len(batch)))
            train_step.run(feed_dict={Q_: tts, x: batch[:,0][0], act: action})
#             print(np.transpose(Q_.eval({Q_: tts})),Qs.eval({x:batch[:,0][0]/6, act: action})[0])
        if episode%10 == 1:
            print(episode,"\t",loss_function.eval({Q_: tts, x:batch[:,0][0], act: action}),"\t",min(Q.eval({x:batch[:,0][0]/6})[0]),"\t", cum_reward, "\t", done,"\t", round(100*done/episode,2))
            


In [ ]:
DQN(c_init=Cube(3),Tmax=1,nb_episodes=100001,n_moves = 1)

1 	 859.123 	 0.0 	 47 	 0 	 0.0
11 	 1526.5 	 -0.285373 	 47 	 0 	 0.0
21 	 945.92 	 0.0795638 	 3 	 0 	 0.0
31 	 37778.0 	 2.92646 	 27 	 1 	 3.23
41 	 30263.4 	 2.76288 	 27 	 1 	 2.44
51 	 23405.3 	 3.40453 	 3 	 1 	 1.96
61 	 19745.2 	 3.39186 	 3 	 1 	 1.64
71 	 35839.8 	 2.65697 	 15 	 2 	 2.82
81 	 29303.2 	 3.81814 	 15 	 2 	 2.47
91 	 38812.8 	 4.38748 	 15 	 3 	 3.3
101 	 47402.7 	 4.8597 	 27 	 4 	 3.96
111 	 58995.0 	 6.8177 	 3 	 5 	 4.5
121 	 58253.4 	 7.8062 	 3 	 7 	 5.79
131 	 47721.3 	 9.39758 	 47 	 7 	 5.34
141 	 26044.7 	 9.5316 	 47 	 7 	 4.96
151 	 48195.0 	 10.0022 	 3 	 7 	 4.64
161 	 47792.7 	 10.0065 	 3 	 9 	 5.59
171 	 58339.4 	 11.1382 	 3 	 10 	 5.85
181 	 70065.2 	 12.3225 	 27 	 10 	 5.52
191 	 80947.6 	 12.7841 	 -1 	 10 	 5.24
201 	 69721.2 	 10.8611 	 3 	 11 	 5.47
211 	 69820.0 	 12.8566 	 27 	 11 	 5.21
221 	 60986.4 	 11.7116 	 3 	 12 	 5.43
231 	 68631.1 	 9.29736 	 -1 	 12 	 5.19
241 	 15987.3 	 10.7016 	 -1 	 12 	 4.98
251 	 49058.5 	 12.1332 

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
correct_prediction

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

In [30]:
data = np.reshape(batch[0][2],[w,h])*255
img = Image.fromarray(data)
img = img.resize((100,100))
img.show()
y.eval({x:batch[0][2].reshape(1,784)})

NameError: name 'batch' is not defined

# CNN

In [2]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [3]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

NameError: name 'x' is not defined

In [6]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

NameError: name 'h_pool1' is not defined

In [7]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

NameError: name 'h_pool2' is not defined

In [8]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

NameError: name 'h_fc1' is not defined

In [9]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

NameError: name 'h_fc1_drop' is not defined

In [10]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

NameError: name 'y_conv' is not defined